# **Dealing with 100+ classes, especially when some are visually similar (like different flower species), requires a more powerful technique than a simple CNN built from scratch. We'll use the Oxford Flowers 102 dataset and implement Transfer Learning using a pre-trained model like MobileNetV2.**



In [19]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [20]:
#  Hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 102 # The Oxford Flowers 102 dataset has 102 classes
AUTOTUNE = tf.data.AUTOTUNE

In [22]:
# Load the Dataset
# Oxford Flowers 102 has 'train', 'validation', and 'test' splits
(train_ds_raw, val_ds_raw, test_ds_raw), metadata = tfds.load(
    'oxford_flowers102',
    split=['train', 'validation', 'test'],
    as_supervised=True,
    with_info=True,
)

print(f"Number of training examples: {metadata.splits['train'].num_examples}")
print(f"Number of classes: {metadata.features['label'].num_classes}")


Number of training examples: 1020
Number of classes: 102


In [23]:
#Preprocessing Function
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [24]:
# Augmentation Layer
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

In [25]:
# Create the Input Pipeline
def prepare_dataset(ds, shuffle=False, augment=False):
    ds = ds.map(preprocess, num_parallel_calls=AUTOTUNE)
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(BATCH_SIZE)
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
    return ds.cache().prefetch(AUTOTUNE)


In [8]:
# Prepare the final datasets
train_ds = prepare_dataset(train_ds_raw, shuffle=True, augment=True)
val_ds = prepare_dataset(val_ds_raw, augment=False)
test_ds = prepare_dataset(test_ds_raw, augment=False)

In [17]:
# --- Load MobileNetV2 Feature Extractor from TF Hub ---
feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_url,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    trainable=False  # freeze the base model
)

# --- Build Model using Functional API ---
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = feature_extractor_layer(inputs)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs, name="mobilenetv2_flowers")

# --- Compile the Model ---
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# --- Model Summary ---
model.summary()

# --- Train the Model ---
EPOCHS = 10
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

# --- Evaluate on Test Dataset ---
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nTest Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# --- Fine-tuning (optional) ---
feature_extractor_layer.trainable = True
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

fine_tune_epochs = 5
total_epochs = EPOCHS + fine_tune_epochs

history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1]
)

# --- Evaluate after Fine-Tuning ---
test_loss, test_acc = model.evaluate(test_ds)
print(f"\nFine-tuned Test Accuracy: {test_acc:.4f}")

# --- Plot Training History ---
acc = history.history['accuracy'] + history_fine.history['accuracy']
val_acc = history.history['val_accuracy'] + history_fine.history['val_accuracy']
loss = history.history['loss'] + history_fine.history['loss']
val_loss = history.history['val_loss'] + history_fine.history['val_loss']

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss')
plt.legend()

plt.show()

# --- Make Predictions on Test Samples ---
class_names = metadata.features['label'].names

for images, labels in test_ds.take(1):
    preds = model.predict(images)
    pred_labels = tf.argmax(preds, axis=1)

    plt.figure(figsize=(12, 12))
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        true_label = class_names[labels[i].numpy()]
        pred_label = class_names[pred_labels[i].numpy()]
        color = "green" if true_label == pred_label else "red"
        plt.title(f"True: {true_label}\nPred: {pred_label}", color=color)
        plt.axis("off")
    plt.show()


ValueError: Exception encountered when calling layer 'keras_layer_7' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'keras_layer_7' (type KerasLayer):
  • inputs=<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, ragged=False, name=keras_tensor_9>
  • training=None